# Over 0.5 HT

**Critérios do Método**

a) Calculamos a Porcentagem de Over 0.5 HT

b) Calculamos a Porcentagem de Over 2.5 FT

c) Calculamos a Média de Gols Total no HT do Mandante

d) Calculamos a Média de Gols Total no HT do Visitante

e) Calculamos o Desvio Padrão e o Coeficiente de Variação dessas Médias

f) Definimos nosso Filtro Estatístico:
1. Porcentagem de Over 0.5 HT Total > 75% 
2. Porcentagem de Over 2.5 FT Total > 50%
3. Média de Gols Total HT Home > 1 
4. Média de Gols Total HT Away > 1
5. CV de Gols Total HT Home < 1 
6. CV de Gols Total HT Away < 1

g) Todos os jogos que possuem esses dados entramos com Odd Mínima de 1.54 




# Importando as Bibliotecas e o DataFrame 

In [ ]:
import pandas as pd
from google.colab import data_table
data_table.enable_dataframe_formatter()
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# DataFrame 2021/2022
# df = pd.read_excel("https://github.com/futpythontrader/YouTube/blob/main/DataFrame_2021_2022.xlsx?raw=true")

# DataFrame 2020/2021
df = pd.read_excel("https://github.com/futpythontrader/YouTube/blob/main/DataFrame_2020_2021.xlsx?raw=true")

# Porcentagem de Gols

In [ ]:
df['Total_Goals_HT'] = df.apply(lambda row: (row['Goals_H_HT'] + row['Goals_A_HT']), axis=1)
df['Total_Goals_FT'] = df.apply(lambda row: (row['Goals_H_FT'] + row['Goals_A_FT']), axis=1) 

df['Over05_HT'] = df.apply(lambda row: 1 if (row['Total_Goals_HT'] > 0) else 0, axis=1) 
df['Over25_FT'] = df.apply(lambda row: 1 if (row['Total_Goals_FT'] > 2) else 0, axis=1) 

In [ ]:
df['Porc_Over05HT_Home'] = (df.groupby('Home')['Over05_HT'].rolling(window=5, min_periods=1).mean().reset_index(0,drop=True) * 100).shift(1)
df['Porc_Over05HT_Away'] = (df.groupby('Away')['Over05_HT'].rolling(window=5, min_periods=1).mean().reset_index(0,drop=True) * 100).shift(1)

df['Porc_Over25FT_Home'] = (df.groupby('Home')['Over25_FT'].rolling(window=5, min_periods=1).mean().reset_index(0,drop=True) * 100).shift(1)
df['Porc_Over25FT_Away'] = (df.groupby('Away')['Over25_FT'].rolling(window=5, min_periods=1).mean().reset_index(0,drop=True) * 100).shift(1)

# Médias, Desvio Padrão e Coeficiente de Variação

In [ ]:
df['Media_Gols_Total_HT_Home'] = df.groupby('Home')['Total_Goals_HT'].rolling(window=5, min_periods=1).mean().reset_index(0,drop=True).shift(1)
df['Media_Gols_Total_HT_Away'] = df.groupby('Away')['Total_Goals_HT'].rolling(window=5, min_periods=1).mean().reset_index(0,drop=True).shift(1)

In [ ]:
df['DP_Gols_Total_HT_Home'] = df.groupby('Home')['Total_Goals_HT'].rolling(window=5, min_periods=1).std().reset_index(0,drop=True).shift(1)
df['DP_Gols_Total_HT_Away'] = df.groupby('Away')['Total_Goals_HT'].rolling(window=5, min_periods=1).std().reset_index(0,drop=True).shift(1)

In [ ]:
df['CV_Gols_Total_HT_Home'] = (df['DP_Gols_Total_HT_Home'] / df['Media_Gols_Total_HT_Home']).shift(1)
df['CV_Gols_Total_HT_Away'] = (df['DP_Gols_Total_HT_Away'] / df['Media_Gols_Total_HT_Away']).shift(1)

In [ ]:
df0 = df.groupby(['Home']).get_group('Liverpool')
df0 = df0[['Home','Away','Goals_H_HT','Goals_A_HT','Media_Gols_Total_HT_Home','Media_Gols_Total_HT_Away','CV_Gols_Total_HT_Home','CV_Gols_Total_HT_Away']]
df0.tail(10)

# Ajustando o DataFrame

In [ ]:
df1 = df[['Home','Away','Goals_H_HT','Goals_A_HT','Total_Goals_HT','Porc_Over05HT_Home','Porc_Over05HT_Away','Porc_Over25FT_Home','Porc_Over25FT_Away','Media_Gols_Total_HT_Home','Media_Gols_Total_HT_Away','CV_Gols_Total_HT_Home','CV_Gols_Total_HT_Away']]
df1 = df1.dropna()

# Filtro Estatístico

In [ ]:
# Selecionando o Filtro Estatítisco
flt2 = ((df1.Porc_Over05HT_Home + df1.Porc_Over05HT_Away) > 75) & ((df1.Porc_Over25FT_Home + df1.Porc_Over25FT_Away) > 50) & (df1.Media_Gols_Total_HT_Home > 1) & (df1.Media_Gols_Total_HT_Away > 1) & (df1.CV_Gols_Total_HT_Home < 1) & (df1.CV_Gols_Total_HT_Away < 1)
# Teste sem o CV
# flt2 = ((df1.Porc_Over05HT_Home + df1.Porc_Over05HT_Away) > 75) & ((df1.Porc_Over25FT_Home + df1.Porc_Over25FT_Away) > 50) & (df1.Media_Gols_Total_HT_Home > 1) & (df1.Media_Gols_Total_HT_Away > 1)
df2 = df1[flt2];
# Ajustando o Índice
df2.reset_index(inplace=True, drop=True)
df2.index = df2.index.set_names(['Nº'])
df2 = df2.rename(index=lambda x: x + 1)

# Resultados

In [ ]:
df2['Odds_Over05_HT'] = 1.54
Stake = 1
green = Stake * (df2.Odds_Over05_HT - 1) 
red = -Stake

# Over 0.5 HT
df2.loc[((df2['Total_Goals_HT']) > 0), 'Profit_Over05HT'] = green
df2.loc[((df2['Total_Goals_HT']) == 0), 'Profit_Over05HT'] = red
df2['Profit_acu_Over05HT'] = df2.Profit_Over05HT.cumsum()
df2.Profit_acu_Over05HT.plot(title='Over 0.5 HT', xlabel='Quantidade de Apostas', ylabel='Stakes')

In [ ]:
df2['Profit_acu_Over05HT'].tail(1)

In [ ]:
df2.head(10)